## 29% using a 29M transformer
Steps to reproduce:  
1. Upload this script to google colab or modal
2. (optional) If you want to save checkpoints and results, mount your google drive (colab) / your volume (modal)
3. Click run-all

This script ensures there's no data contamination. 
This produces a `submission.json` file. The `submission.json` file


Notes:
1. The config in this notebook has been tuned for an 80GB A100  
2. Actual results were obtained by running this exact file in 2 phases.  
    - Training on a 40GB A100
    - Take the final checkpoint, and run the inference on an 80GB A100

This will work on smaller GPUs too, but will take longer to train  
For very constrained environments, disable the "do_validate" flag. This avoids checking the validation loss every epoch

In [ ]:
# root_folder, mount_folder = "root", "mnt/mithil-arc" # for modal - REPLACE /mithil-arc WITH YOUR VOLUME NAME
root_folder, mount_folder = "content", "content/drive/MyDrive"  # for colab

%cd /$root_folder/
!git clone https://github.com/mvakde/mdlARC.git # `-b <branch_name> --single-branch` if branch
%cd /$root_folder/mdlARC

In [ ]:
!python dataset_building_scripts/build_datasets.py --datasets arc1 conceptarc  --splits train eval --with-solutions --cleanup none
!python dataset_building_scripts/augment_dataset_dihedral.py

!rm -rf /$root_folder/mdlARC/run-script.ipynb
!rm -rf /$root_folder/mdlARC/sanitised-env-run-script.ipynb
!rm -rf /$root_folder/mdlARC/ultra-sanitised-env-run-script.ipynb
!rm -rf /$root_folder/mdlARC/dataset_building_scripts
!rm -rf /$root_folder/mdlARC/readme.md
!rm -rf /$root_folder/mdlARC/img

In [ ]:
from pathlib import Path
import argparse
import importlib
import sys

PROJECT_ROOT = Path.cwd()
SRC_DIR = PROJECT_ROOT / "src"
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

import utils, tinytransformer, train

importlib.reload(utils)  # pick up code changes during iteration
importlib.reload(tinytransformer)
importlib.reload(train)

args = {
    # run config
    "num_workers": 0,
    "device": "cuda",  # 'cuda' | 'mps' | 'cpu'
    "do_validate": False,
    "name": "arc1-cleanenv-30M-vvwide-bs32-101ep-100color-ccdb-18dec0430",  # download file name
    "GPU": "A100-noaugreg",  # just for logging purposes
    # paths - must pass as Path("<path_to_dir>")
    "train_log_file": Path("runs/training_log.txt"),
    "save_path": Path("runs/tiny.pt"),
    "checkpoint_path": None,  # Path("runs/tiny.pt"),  # or None to start from scratch
    "data_path": Path("assets/challenges_dihedral_both.json"),
    # hyperparameters
    "epochs": 101,
    "batch_size": 32,
    "val_batch_size": 300,
    "enable_color_aug_train": True,
    "max_color_augments_train": 100,
    "color_aug_seed": 42,
    "lr": 3e-4,
    "weight_decay": 0.01,
    "grad_clip": 1.0,
    "dropout": 0.1,
    "seed": 42,
    # Model Architecture
    "d_model": 768,  # 128, 256, 512, 768 | 128, 384, 640
    "n_heads": 12,  # 4, 8, 8/16, 12 | 4, 12, 10
    "d_ff": 3072,  # 512, 1024, 2048, 3072 | 512, 1536, 2560
    "n_layers": 4,  # 4, 6, 16, 16 | 24, 28, 24
    # Visibility toggles
    "log_train_strings": False,
    "log_train_limit": 10,
    "log_inference_prompt": False,
    "inference_temperature": None,
    "inference_top_k": None,
}
cfg = argparse.Namespace(**args)

runs_dir = Path("runs")
runs_dir.mkdir(parents=True, exist_ok=True)
with (runs_dir / "config.txt").open("w") as f:
    for k, v in args.items():
        f.write(f"{k}: {v}\n")

model, dataset, dataloader, device, data_path = train.build_model_and_data(cfg)

In [ ]:
# Training only

from time import perf_counter

t_start = perf_counter()
train.train_model(cfg,model=model,dataloader=dataloader,dataset=dataset,device=device,data_path=data_path)
t_duration = perf_counter() - t_start

print(f"Training took {t_duration:.2f}s")
with open(Path("runs/timing.txt"), "w") as f:
    f.write(f"Training: {t_duration:.4f} s\n")

In [ ]:
# cleaning up memory to run inference
utils.cleanup_memory(globals())


In [ ]:
# save data immediately in case eval fails
archive_state = utils.save_run_archive(
    cfg.name, root_folder, mount_folder, globals_dict=globals()
)


In [ ]:
from pathlib import Path
import importlib
import evaluations
import utils
importlib.reload(evaluations)
importlib.reload(utils)

PATH_BOTH = Path("assets/challenges_dihedral_both.json")

EVAL_CONFIGS = [
    # ("eval_125color_both", 125, PATH_BOTH),
    ("eval_100color_both", 100, PATH_BOTH),
    # ("eval_10color_both", 10, PATH_BOTH),
    # ("eval_0color_both", 0, PATH_BOTH),
    # ("eval_0color_train", 0, PATH_TRAIN)  # Uses TRAIN path (No Geom TTA on Test)
]

EVAL_BATCH_SIZE = 1300
SPLITS = ["test"]
CHECKPOINT_PATH = Path("runs/tiny.pt")
SOLUTIONS_PRESENT = False
EVAL_TASK_IDS = None  # Set to None to evaluate full dataset, or ["00576224", ...] for specific tasks
LOG_CORRECT_GRIDS = False  # Print the actual grid, IDs, and augmentation indices for fully correct grids

evaluations.run_evaluation_configs(
    cfg,
    EVAL_CONFIGS,
    eval_batch_size=EVAL_BATCH_SIZE,
    splits=SPLITS,
    checkpoint_path=CHECKPOINT_PATH,
    include_targets=SOLUTIONS_PRESENT,
    task_ids=EVAL_TASK_IDS,
    log_correct_grids=LOG_CORRECT_GRIDS,
)


In [ ]:
# refresh Drive zip
archive_state = utils.update_run_archive(
    cfg.name, root_folder, mount_folder, globals_dict=globals()
)


In [ ]:
# visualisation
EVAL_SUB_FOLDER = "eval_100color_both"
VIS_MODE = "!"  # "!" = compare vs solutions, "submission" = attempts-only
utils.visualize_eval_submissions(
    EVAL_SUB_FOLDER, mode=VIS_MODE, solutions_file="assets/solutions.json"
)
